In [1]:
num_rollouts = 100
num_workers = 15
yaml_path = "data/ablation_study_180_agents_0831/ablation_study_180_agents_0831.yaml"

In [2]:
from process_fft import get_fft_cluster_finder
from process_cluster import load_cluster_df
from reduce_dimension import *
import matplotlib.pyplot as plt
import seaborn as sns
from record_video import generate_video_of_cluster

W0831 20:39:28.446441 139621305218816 deprecation.py:323] From /home/zhpeng/anaconda3/envs/uber/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
ret = get_fft_cluster_finder(
    yaml_path=yaml_path,
    num_rollouts=num_rollouts,
    num_workers=num_workers
)
cluster_finder = ret['cluster_finder']['nostd_cluster_finder']
prefix = ret['prefix']

Successfully loaded name_ckpt_mapping!
Sucessfully initialize Ray!
[1/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=187 rew=282.21>
[2/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=98 rew=282.24>
[3/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=203 rew=282.25>
[4/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=197 rew=282.48>
[5/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=71 rew=282.55>
[6/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=283 rew=282.79>
[7/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=116 rew=283.13>
[8/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=70 rew=283.20>
[9/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=201 rew=283.28>
[10/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=113 rew=283.29>
[11/180] (+0.0s/0.0s) Start collecting data from agent <PPO seed=7 rew=283.41>
[12/180] (+0.0s/0.0s) Start collecting data from agent <

In [ ]:
num_clusters = 10
cluster_finder.set(num_clusters)

In [ ]:
prediction = cluster_finder.predict()
print(
    "Collected clustering results for {} agents, {} clusters.".format(
        len(prediction), num_clusters
    )
)

In [ ]:
cluster_df = load_cluster_df(prefix + ".pkl")

In [ ]:
plot_df, _ = reduce_dimension(cluster_df, prediction, False)

In [ ]:
def get_label(name):
    if name.startswith("ES"):
        return "ES"
    if "fc2" in name:
        return "fc2"
    if "fc_out" in name:
        return "fc_out"
    else:
        return "PPO"
    
plot_df.insert(4, "label", [get_label(name) for name in plot_df.agent], True)

In [ ]:
save = prefix + "_2d.png"
show = None

def _get_title(plot_df):
    num_clusters = len(plot_df.cluster.unique())
    num_agents = len(plot_df.agent.unique())
    return "Clustering Result of {} Clusters, " \
           "{} Agents (Dimensions Reduced by PCA-TSNE)".format(
        num_clusters, num_agents)


plt.figure(figsize=(12, 10), dpi=300)
num_clusters = len(plot_df.cluster.unique())
palette = sns.color_palette(n_colors=num_clusters)
ax = sns.scatterplot(
    x="x",
    y="y",
    hue="cluster",
    style="label",
    palette=palette,
    data=plot_df,
    legend="full"
)
ax.set_title(_get_title(plot_df))
if save is not None:
    assert save.endswith('png')
    plt.savefig(save, dpi=300)
if show:
    plt.show()

In [ ]:

# generate grid of videos with shape (k, max_num_cols)
generate_video_of_cluster(
    prediction=prediction,
    num_agents=None,
    yaml_path=yaml_path,
    video_prefix=prefix,
    max_num_cols=18,
    num_workers=num_workers
)
print("Finished generating videos.")
